<a href="https://colab.research.google.com/github/qweliant/DS-Unit-2-Regression-Classification/blob/master/assignment_regression_classification_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science, Unit 2: Predictive Modeling

# Regression & Classification, Module 3

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

Instead, predict property sales prices for **One Family Dwellings** (`BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'`). 

Use a subset of the data where the **sale price was more than \\$100 thousand and less than $2 million.** 

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.

- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Fit a ridge regression model with multiple features.
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.


## Stretch Goals
- [ ] Add your own stretch goal(s) !
- [ ] Instead of `RidgeRegression`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `RidgeRegression`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

# New Section

In [1]:
import os, sys
in_colab = 'google.colab' in sys.modules

# If you're in Colab...
if in_colab:
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification.git
    !git pull origin master
    
    # Install required python packages
    !pip install -r requirements.txt
    
    # Change into directory for module
    os.chdir('module3')

Reinitialized existing Git repository in /content/.git/
fatal: remote origin already exists.
From https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification
 * branch            master     -> FETCH_HEAD
Already up to date.


In [0]:
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv('../data/condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'
df.SALE_DATE = pd.to_datetime(df.SALE_DATE)
df2 = df

In [6]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,2019-01-01
1,1,OTHER,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,2019-01-01
2,1,OTHER,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,2019-01-01
3,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,2019-01-01
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,2019-01-01


In [7]:
df.describe()

,BLOCK,LOT,EASE-MENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,SALE_PRICE
count,23040.000000,23040.000000,0.0,23039.000000,23039.000000,23039.000000,23039.000000,2.303900e+04,23005.000000,23040.000000,2.304000e+04
mean,4459.116493,353.948481,NaN,10791.186987,1.827423,0.286514,2.320891,3.508555e+03,1819.671724,1.624479,1.328133e+06
std,3713.269887,628.025255,NaN,1103.414210,10.090432,5.778224,12.070554,2.218610e+04,488.376571,0.803898,1.025394e+07
min,1.000000,1.000000,NaN,0.000000,0.000000,-148.000000,0.000000,0.000000e+00,0.000000,1.000000,0.000000e+00
25%,1341.000000,22.000000,NaN,10306.000000,0.000000,0.000000,1.000000,4.940000e+02,1920.000000,1.000000,0.000000e+00
50%,3546.000000,49.000000,NaN,11211.000000,1.000000,0.000000,1.000000,1.356000e+03,1940.000000,1.000000,4.550000e+05
75%,6673.750000,375.000000,NaN,11360.000000,2.000000,0.000000,2.000000,2.280000e+03,1965.000000,2.000000,8.750000e+05
max,16350.000000,9057.000000,NaN,11697.000000,750.000000,570.000000,755.000000,1.303935e+06,2019.000000,4.000000,8.500000e+08


In [8]:
df.shape

(23040, 21)

In [0]:

cut = ((df.SALE_PRICE > 100000) & (df.SALE_PRICE < 2000000) & (df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS'))



In [10]:
df2 = df2[cut] 
print(df2.shape)
df2.head()

(3151, 21)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,2019-01-01
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000,2019-01-01
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,NaN,A1,1260 RHINELANDER AVE,NaN,10461.0,1.0,0.0,1.0,"3,500",2043.0,1925.0,1,A1,810000,2019-01-02
108,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,NaN,A1,469 E 25TH ST,NaN,11226.0,1.0,0.0,1.0,"4,000",2680.0,1899.0,1,A1,125000,2019-01-02
111,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,NaN,A5,5521 WHITTY LANE,NaN,11203.0,1.0,0.0,1.0,"1,710",1872.0,1940.0,1,A5,620000,2019-01-02


In [11]:
df2.LAND_SQUARE_FEET = [ float(i.replace(',','')) for i in df2.LAND_SQUARE_FEET]
df2.dtypes


/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


BOROUGH                                   object
NEIGHBORHOOD                              object
BUILDING_CLASS_CATEGORY                   object
TAX_CLASS_AT_PRESENT                      object
BLOCK                                      int64
LOT                                        int64
EASE-MENT                                float64
BUILDING_CLASS_AT_PRESENT                 object
ADDRESS                                   object
APARTMENT_NUMBER                          object
ZIP_CODE                                 float64
RESIDENTIAL_UNITS                        float64
COMMERCIAL_UNITS                         float64
TOTAL_UNITS                              float64
LAND_SQUARE_FEET                         float64
GROSS_SQUARE_FEET                        float64
YEAR_BUILT                               float64
TAX_CLASS_AT_TIME_OF_SALE                  int64
BUILDING_CLASS_AT_TIME_OF_SALE            object
SALE_PRICE                                 int64
SALE_DATE           

In [12]:
print(df2.shape)
df2.head()

(3151, 21)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,6800.0,1325.0,1930.0,1,A9,550000,2019-01-01
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,4000.0,2001.0,1940.0,1,A1,200000,2019-01-01
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,NaN,A1,1260 RHINELANDER AVE,NaN,10461.0,1.0,0.0,1.0,3500.0,2043.0,1925.0,1,A1,810000,2019-01-02
108,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,NaN,A1,469 E 25TH ST,NaN,11226.0,1.0,0.0,1.0,4000.0,2680.0,1899.0,1,A1,125000,2019-01-02
111,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,NaN,A5,5521 WHITTY LANE,NaN,11203.0,1.0,0.0,1.0,1710.0,1872.0,1940.0,1,A5,620000,2019-01-02


In [13]:
# drop becasue it goes hand in hand with BUILDING_CLASS_CATEGORY, EASE-MENT becasue its mostly NaNs
df2.drop(columns=['TAX_CLASS_AT_PRESENT', 'EASE-MENT', 'APARTMENT_NUMBER'], inplace=True)


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [14]:
df2.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ADDRESS,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,5495,801,A9,4832 BAY PARKWAY,11230.0,1.0,0.0,1.0,6800.0,1325.0,1930.0,1,A9,550000,2019-01-01
61,4,OTHER,01 ONE FAMILY DWELLINGS,7918,72,A1,80-23 232ND STREET,11427.0,1.0,0.0,1.0,4000.0,2001.0,1940.0,1,A1,200000,2019-01-01
78,2,OTHER,01 ONE FAMILY DWELLINGS,4210,19,A1,1260 RHINELANDER AVE,10461.0,1.0,0.0,1.0,3500.0,2043.0,1925.0,1,A1,810000,2019-01-02
108,3,OTHER,01 ONE FAMILY DWELLINGS,5212,69,A1,469 E 25TH ST,11226.0,1.0,0.0,1.0,4000.0,2680.0,1899.0,1,A1,125000,2019-01-02
111,3,OTHER,01 ONE FAMILY DWELLINGS,7930,121,A5,5521 WHITTY LANE,11203.0,1.0,0.0,1.0,1710.0,1872.0,1940.0,1,A5,620000,2019-01-02


In [15]:
print(df2.BOROUGH.unique(),'\n')
print(df2.NEIGHBORHOOD.unique(), '\n')

print(df2.BUILDING_CLASS_AT_PRESENT.unique(), '\n')
print(df2.BUILDING_CLASS_AT_TIME_OF_SALE.unique(), '\n')
print(df2.ZIP_CODE.unique() , '\n')

['3' '4' '2' '5' '1'] 

['OTHER' 'FLUSHING-NORTH' 'EAST NEW YORK' 'BEDFORD STUYVESANT'
 'FOREST HILLS' 'BOROUGH PARK' 'ASTORIA'] 

['A9' 'A1' 'A5' 'A0' 'A2' 'A3' 'S1' 'A4' 'A6' 'A8' 'B2' 'S0' 'B3'] 

['A9' 'A1' 'A5' 'A0' 'A2' 'A3' 'S1' 'A4' 'A6' 'A8' 'S0'] 

[11230. 11427. 10461. 11226. 11203. 11229. 11364. 11373. 11365. 11429.
 11414. 11369. 11415. 11413. 11434. 11435. 10312. 10308. 10314. 11236.
 11228. 11207. 11218. 11210. 11234. 11411. 11362. 11691. 11354. 11367.
 11366. 11423. 11432. 11385. 11420. 10304. 10306. 10302. 10305. 10309.
 10469. 10463. 11216. 11215. 11360. 11361. 11426. 11375. 11363. 11379.
 11374. 11694. 11422. 11433. 11104. 11357. 10307. 10466. 10473. 11209.
 11428. 11417. 11418. 11412. 10303. 10457. 10454. 10465. 11223. 11697.
 11040. 11419. 11377. 10452. 11356. 11358. 11421. 10472. 11204. 11208.
 11355. 11378. 10467. 11212. 11370. 10301. 10310. 10471. 11001. 11693.
 11219. 11235. 11436. 10470. 11105. 10475. 11220. 11213. 11214. 11103.
 11004. 11217. 11224. 10453. 11

In [16]:
df2.replace({'A3':1, 'A7':2, 'A9':3, 'A4':4, 'A1':5, 'S1':6, 'S0':7, 'A5':8}, inplace =True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)


In [17]:
df2.NEIGHBORHOOD.value_counts()

OTHER                 2959
FLUSHING-NORTH          97
EAST NEW YORK           31
FOREST HILLS            22
BOROUGH PARK            19
ASTORIA                 14
BEDFORD STUYVESANT       9
Name: NEIGHBORHOOD, dtype: int64

In [18]:
df2.NEIGHBORHOOD.replace({'OTHER': 1, 'UPPER EAST SIDE (59-79)': 2, 'FOREST HILLS':3,  'UPPER EAST SIDE (79-96)':4, 'UPPER WEST SIDE (79-96)':5,  'UPPER EAST SIDE (79-96)':6 , 'BEDFORD STUYVESANT':7}, inplace=True)
df2.NEIGHBORHOOD.value_counts()

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


1                 2959
FLUSHING-NORTH      97
EAST NEW YORK       31
3                   22
BOROUGH PARK        19
ASTORIA             14
7                    9
Name: NEIGHBORHOOD, dtype: int64

In [19]:
df2.head(3)

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ADDRESS,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,1,01 ONE FAMILY DWELLINGS,5495,801,3,4832 BAY PARKWAY,11230.0,1.0,0.0,1.0,6800.0,1325.0,1930.0,1,3,550000,2019-01-01
61,4,1,01 ONE FAMILY DWELLINGS,7918,72,5,80-23 232ND STREET,11427.0,1.0,0.0,1.0,4000.0,2001.0,1940.0,1,5,200000,2019-01-01
78,2,1,01 ONE FAMILY DWELLINGS,4210,19,5,1260 RHINELANDER AVE,10461.0,1.0,0.0,1.0,3500.0,2043.0,1925.0,1,5,810000,2019-01-02


In [20]:
cutoff = pd.to_datetime('2019-04-01')
train = df2[df2.SALE_DATE < cutoff]
test  = df2[df2.SALE_DATE >= cutoff]
train.shape, test.shape

((2507, 18), (644, 18))

In [0]:
import pandas_profiling


In [22]:
train.profile_report()


In [23]:
train.describe(exclude='number').T.sort_values(by='unique')

,count,unique,top,freq,first,last
BUILDING_CLASS_CATEGORY,2507,1,01 ONE FAMILY DWELLINGS,2507,NaN,NaN
BOROUGH,2507,5,4,1204,NaN,NaN
NEIGHBORHOOD,2507,7,1,2360,NaN,NaN
BUILDING_CLASS_AT_TIME_OF_SALE,2507,11,5,919,NaN,NaN
BUILDING_CLASS_AT_PRESENT,2507,13,5,919,NaN,NaN
SALE_DATE,2507,68,2019-01-31 00:00:00,78,2019-01-01 00:00:00,2019-03-30 00:00:00
ADDRESS,2507,2497,118-20 202ND STREET,2,NaN,NaN


In [0]:
target = 'SALE_PRICE'
high_cardinality  = ['SALE_DATE', 'ADDRESS']
feature = train.columns.drop([target] + high_cardinality)

X_train = train[feature]
y_train = train[target]
X_test = test[feature]
y_test = test[target]

In [25]:
X_train.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE
44,3,1,01 ONE FAMILY DWELLINGS,5495,801,3,11230.0,1.0,0.0,1.0,6800.0,1325.0,1930.0,1,3
61,4,1,01 ONE FAMILY DWELLINGS,7918,72,5,11427.0,1.0,0.0,1.0,4000.0,2001.0,1940.0,1,5
78,2,1,01 ONE FAMILY DWELLINGS,4210,19,5,10461.0,1.0,0.0,1.0,3500.0,2043.0,1925.0,1,5
108,3,1,01 ONE FAMILY DWELLINGS,5212,69,5,11226.0,1.0,0.0,1.0,4000.0,2680.0,1899.0,1,5
111,3,1,01 ONE FAMILY DWELLINGS,7930,121,8,11203.0,1.0,0.0,1.0,1710.0,1872.0,1940.0,1,8


In [0]:
import category_encoders as ce
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

In [27]:
X_train.head()

,BOROUGH_3,BOROUGH_4,BOROUGH_2,BOROUGH_5,BOROUGH_1,NEIGHBORHOOD_1,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_EAST NEW YORK,NEIGHBORHOOD_7,NEIGHBORHOOD_3,NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_ASTORIA,BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT_3,BUILDING_CLASS_AT_PRESENT_5,BUILDING_CLASS_AT_PRESENT_8,BUILDING_CLASS_AT_PRESENT_A0,BUILDING_CLASS_AT_PRESENT_A2,BUILDING_CLASS_AT_PRESENT_1,BUILDING_CLASS_AT_PRESENT_6,BUILDING_CLASS_AT_PRESENT_4,BUILDING_CLASS_AT_PRESENT_A6,BUILDING_CLASS_AT_PRESENT_A8,BUILDING_CLASS_AT_PRESENT_B2,BUILDING_CLASS_AT_PRESENT_7,BUILDING_CLASS_AT_PRESENT_B3,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE_3,BUILDING_CLASS_AT_TIME_OF_SALE_5,BUILDING_CLASS_AT_TIME_OF_SALE_8,BUILDING_CLASS_AT_TIME_OF_SALE_A0,BUILDING_CLASS_AT_TIME_OF_SALE_A2,BUILDING_CLASS_AT_TIME_OF_SALE_1,BUILDING_CLASS_AT_TIME_OF_SALE_6,BUILDING_CLASS_AT_TIME_OF_SALE_4,BUILDING_CLASS_AT_TIME_OF_SALE_A6,BUILDING_CLASS_AT_TIME_OF_SALE_A8,BUILDING_CLASS_AT_TIME_OF_SALE_7
44,1,0,0,0,0,1,0,0,0,0,0,0,1,5495,801,1,0,0,0,0,0,0,0,0,0,0,0,0,11230.0,1.0,0.0,1.0,6800.0,1325.0,1930.0,1,1,0,0,0,0,0,0,0,0,0,0
61,0,1,0,0,0,1,0,0,0,0,0,0,1,7918,72,0,1,0,0,0,0,0,0,0,0,0,0,0,11427.0,1.0,0.0,1.0,4000.0,2001.0,1940.0,1,0,1,0,0,0,0,0,0,0,0,0
78,0,0,1,0,0,1,0,0,0,0,0,0,1,4210,19,0,1,0,0,0,0,0,0,0,0,0,0,0,10461.0,1.0,0.0,1.0,3500.0,2043.0,1925.0,1,0,1,0,0,0,0,0,0,0,0,0
108,1,0,0,0,0,1,0,0,0,0,0,0,1,5212,69,0,1,0,0,0,0,0,0,0,0,0,0,0,11226.0,1.0,0.0,1.0,4000.0,2680.0,1899.0,1,0,1,0,0,0,0,0,0,0,0,0
111,1,0,0,0,0,1,0,0,0,0,0,0,1,7930,121,0,0,1,0,0,0,0,0,0,0,0,0,0,11203.0,1.0,0.0,1.0,1710.0,1872.0,1940.0,1,0,0,1,0,0,0,0,0,0,0,0


In [0]:
warnings.filterwarnings(action='ignore', category=RuntimeWarning, module='sklearn')
warnings.filterwarnings(action='ignore', category=RuntimeWarning, module='scipy')

In [0]:
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression


In [30]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_test_scaled.shape, X_train_scaled

((644, 47), array([[ 2.3019546 , -0.96126036, -0.32686888, ..., -0.07493813,
         -0.11189367, -0.01997604],
        [-0.43441343,  1.04030088, -0.32686888, ..., -0.07493813,
         -0.11189367, -0.01997604],
        [-0.43441343, -0.96126036,  3.05933067, ..., -0.07493813,
         -0.11189367, -0.01997604],
        ...,
        [-0.43441343, -0.96126036, -0.32686888, ..., -0.07493813,
         -0.11189367, -0.01997604],
        [-0.43441343, -0.96126036, -0.32686888, ..., -0.07493813,
         -0.11189367, -0.01997604],
        [-0.43441343,  1.04030088, -0.32686888, ..., -0.07493813,
         -0.11189367, -0.01997604]]))

In [31]:
for k in range(1, len(X_train.columns)+1):
    
    print(f'{k} features')
    
    selector = SelectKBest(score_func=f_regression, k=k)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)
    
    model = LinearRegression()
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)
    
    mae = mean_absolute_error(y_test, y_pred)
    print(f'Test MAE: ${mae:,.0f} \n')

1 features
Test MAE: $183,641 

2 features
Test MAE: $179,555 

3 features
Test MAE: $179,291 

4 features
Test MAE: $179,291 

5 features
Test MAE: $178,897 

6 features
Test MAE: $177,128 

7 features
Test MAE: $171,378 

8 features
Test MAE: $171,465 

9 features
Test MAE: $170,925 

10 features
Test MAE: $167,468 

11 features
Test MAE: $167,182 

12 features
Test MAE: $166,712 

13 features
Test MAE: $156,488 

14 features
Test MAE: $157,546 

15 features
Test MAE: $157,543 

16 features
Test MAE: $157,545 

17 features
Test MAE: $157,406 

18 features
Test MAE: $157,406 

19 features
Test MAE: $157,282 

20 features
Test MAE: $157,282 

21 features
Test MAE: $152,830 

22 features
Test MAE: $152,861 

23 features
Test MAE: $152,861 

24 features
Test MAE: $152,861 

25 features
Test MAE: $152,861 

26 features
Test MAE: $152,552 

27 features
Test MAE: $152,415 

28 features
Test MAE: $153,415 

29 features
Test MAE: $153,570 

30 features
Test MAE: $153,570 

31 features
Test MA

In [32]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

for k in range(1, len(X_train.columns)+1):
    print(f'{k} features')
    
    selector = SelectKBest(score_func=f_regression, k=k)
    X_train_selected = selector.fit_transform(X_train_scaled, y_train)
    X_test_selected = selector.transform(X_test_scaled)
    
    model = RidgeCV()
    model.fit(X_train_selected, y_train)
    
    y_pred = model.predict(X_test_selected)
    mae = mean_absolute_error(y_test, y_pred)
    print(f'Test MAE: ${mae:,.0f} \n')
    


1 features
Test MAE: $183,670 

2 features
Test MAE: $179,587 

3 features
Test MAE: $179,321 

4 features
Test MAE: $179,321 

5 features
Test MAE: $178,901 

6 features
Test MAE: $177,104 

7 features
Test MAE: $171,375 

8 features
Test MAE: $171,440 

9 features
Test MAE: $170,898 

10 features
Test MAE: $167,490 

11 features
Test MAE: $167,181 

12 features
Test MAE: $166,721 

13 features
Test MAE: $156,454 

14 features
Test MAE: $157,503 

15 features
Test MAE: $157,500 

16 features
Test MAE: $157,502 

17 features
Test MAE: $157,364 

18 features
Test MAE: $157,363 

19 features
Test MAE: $157,239 

20 features
Test MAE: $157,238 

21 features
Test MAE: $152,793 

22 features
Test MAE: $152,616 

23 features
Test MAE: $152,641 

24 features
Test MAE: $152,642 

25 features
Test MAE: $152,418 

26 features
Test MAE: $152,596 

27 features
Test MAE: $152,499 

28 features
Test MAE: $153,500 

29 features
Test MAE: $153,654 

30 features
Test MAE: $153,654 

31 features
Test MA

In [33]:
# Which features were selected?
all = X_train.columns
mask = selector.get_support()
select_names = all[mask]
unselect_names = all[~mask]

print('Features selected:')
for name in select_names:
    print(name)
    
print('\n')
print('Features not selected:')
for name in unselect_names:
    print(name)

Features selected:
BOROUGH_3
BOROUGH_4
BOROUGH_2
BOROUGH_5
BOROUGH_1
NEIGHBORHOOD_1
NEIGHBORHOOD_FLUSHING-NORTH
NEIGHBORHOOD_EAST NEW YORK
NEIGHBORHOOD_7
NEIGHBORHOOD_3
NEIGHBORHOOD_BOROUGH PARK
NEIGHBORHOOD_ASTORIA
BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS
BLOCK
LOT
BUILDING_CLASS_AT_PRESENT_3
BUILDING_CLASS_AT_PRESENT_5
BUILDING_CLASS_AT_PRESENT_8
BUILDING_CLASS_AT_PRESENT_A0
BUILDING_CLASS_AT_PRESENT_A2
BUILDING_CLASS_AT_PRESENT_1
BUILDING_CLASS_AT_PRESENT_6
BUILDING_CLASS_AT_PRESENT_4
BUILDING_CLASS_AT_PRESENT_A6
BUILDING_CLASS_AT_PRESENT_A8
BUILDING_CLASS_AT_PRESENT_B2
BUILDING_CLASS_AT_PRESENT_7
BUILDING_CLASS_AT_PRESENT_B3
ZIP_CODE
RESIDENTIAL_UNITS
COMMERCIAL_UNITS
TOTAL_UNITS
LAND_SQUARE_FEET
GROSS_SQUARE_FEET
YEAR_BUILT
TAX_CLASS_AT_TIME_OF_SALE
BUILDING_CLASS_AT_TIME_OF_SALE_3
BUILDING_CLASS_AT_TIME_OF_SALE_5
BUILDING_CLASS_AT_TIME_OF_SALE_8
BUILDING_CLASS_AT_TIME_OF_SALE_A0
BUILDING_CLASS_AT_TIME_OF_SALE_A2
BUILDING_CLASS_AT_TIME_OF_SALE_1
BUILDING_CLASS_AT_TIME_OF_SA